<a href="https://colab.research.google.com/github/entenam/TF-Intro/blob/main/Cat-v-DogsDataAugmentationCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

**IMPORTANT NOTE:** This notebook is designed to run as a Colab. Click the button on top that says, `Open in Colab`, to run this notebook as a Colab. Running the notebook on your local machine might result in some of the code blocks throwing errors.

In [1]:
# In this exercise you will train a CNN on the FULL Cats-v-dogs dataset
# This will require you doing a lot of data preprocessing because
# the dataset isn't split into training and validation for you
# This code block has all the required inputs
import os
import zipfile
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile

In [3]:
# This code block downloads the full Cats-v-Dogs dataset and stores it as 
# cats-and-dogs.zip. It then unzips it to /tmp
# which will create a tmp/PetImages directory containing subdirectories
# called 'Cat' and 'Dog' (that's how the original researchers structured it)
# If the URL doesn't work, 
# .   visit https://www.microsoft.com/en-us/download/confirmation.aspx?id=54765
# And right click on the 'Download Manually' link to get a new URL

!wget --no-check-certificate \
    "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip" \
    -O "/content/drive/MyDrive/AI Basics with Tensorflow/CATS-v-DOGS/cats-and-dogs.zip"

local_zip = '/content/drive/MyDrive/AI Basics with Tensorflow/CATS-v-DOGS/cats-and-dogs.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/drive/MyDrive/AI Basics with Tensorflow/CATS-v-DOGS')
zip_ref.close()


--2022-01-17 08:16:11--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
Resolving download.microsoft.com (download.microsoft.com)... 96.17.56.116, 2600:1407:3c00:10a2::e59, 2600:1407:3c00:108c::e59
Connecting to download.microsoft.com (download.microsoft.com)|96.17.56.116|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824894548 (787M) [application/octet-stream]
Saving to: ‘/content/drive/MyDrive/AI Basics with Tensorflow/CATS-v-DOGS/cats-and-dogs.zip’

/content/drive/MyDr 100%[===================>] 786.68M  34.4MB/s    in 23s     

2022-01-17 08:16:34 (33.9 MB/s) - ‘/content/drive/MyDrive/AI Basics with Tensorflow/CATS-v-DOGS/cats-and-dogs.zip’ saved [824894548/824894548]



In [30]:
print(len(os.listdir('/content/drive/MyDrive/AI Basics with Tensorflow/CATS-v-DOGS/PetImages/Cat/')))
print(len(os.listdir('/content/drive/MyDrive/AI Basics with Tensorflow/CATS-v-DOGS/PetImages/Dog/')))

# Expected Output:
# 12501
# 12501

12501
12501


In [39]:
# Use os.mkdir to create your directories
# You will need a directory for cats-v-dogs, and subdirectories for training
# and testing. These in turn will need subdirectories for 'cats' and 'dogs'
try:
    classes = ['cats','dogs']
  
    for class_name in classes:
      ### START CODE HERE
      os.mkdir("/content/drive/MyDrive/AI Basics with Tensorflow/CATS-v-DOGS/cats-v-dogs")
      os.mkdir("/content/drive/MyDrive/AI Basics with Tensorflow/CATS-v-DOGS/cats-v-dogs/training")
      os.mkdir("/content/drive/MyDrive/AI Basics with Tensorflow/CATS-v-DOGS/cats-v-dogs/testing")
      os.mkdir("/content/drive/MyDrive/AI Basics with Tensorflow/CATS-v-DOGS/cats-v-dogs/training/cats")
      os.mkdir("/content/drive/MyDrive/AI Basics with Tensorflow/CATS-v-DOGS/cats-v-dogs/testing/cats")
      os.mkdir("/content/drive/MyDrive/AI Basics with Tensorflow/CATS-v-DOGS/cats-v-dogs/training/dogs")
      os.mkdir("/content/drive/MyDrive/AI Basics with Tensorflow/CATS-v-DOGS/cats-v-dogs/testing/dogs")
        
      ### END CODE HERE
except OSError:
    pass

In [40]:
# Write a python function called split_data which takes
# a SOURCE directory containing the files
# a TRAINING directory that a portion of the files will be copied to
# a TESTING directory that a portion of the files will be copie to
# a SPLIT SIZE to determine the portion
# The files should also be randomized, so that the training set is a random
# X% of the files, and the test set is the remaining files
# SO, for example, if SOURCE is PetImages/Cat, and SPLIT SIZE is .9
# Then 90% of the images in PetImages/Cat will be copied to the TRAINING dir
# and 10% of the images will be copied to the TESTING dir
# Also -- All images should be checked, and if they have a zero file length,
# they will not be copied over
#
# os.listdir(DIRECTORY) gives you a listing of the contents of that directory
# os.path.getsize(PATH) gives you the size of the file
# copyfile(source, destination) copies a file from source to destination
# random.sample(list, len(list)) shuffles a list
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
  
  ### START CODE HERE
  for filename in os.listdir(SOURCE):
    if os.path.getsize(os.path.join(SOURCE, filename))>0:
      if random.uniform(0, 1) <= SPLIT_SIZE:
        copyfile(os.path.join(SOURCE, filename), os.path.join(TRAINING, filename))
      else:
        copyfile(os.path.join(SOURCE, filename), os.path.join(TESTING, filename))
    else:
      print(filename + " is zero length, so ignoring")

  ### END CODE HERE


CAT_SOURCE_DIR = "/content/drive/MyDrive/AI Basics with Tensorflow/CATS-v-DOGS/PetImages/Cat/"
TRAINING_CATS_DIR = "/content/drive/MyDrive/AI Basics with Tensorflow/CATS-v-DOGS/cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "/content/drive/MyDrive/AI Basics with Tensorflow/CATS-v-DOGS/cats-v-dogs/testing/cats/"
DOG_SOURCE_DIR = "/content/drive/MyDrive/AI Basics with Tensorflow/CATS-v-DOGS/PetImages/Dog/"
TRAINING_DOGS_DIR = "/content/drive/MyDrive/AI Basics with Tensorflow/CATS-v-DOGS/cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "/content/drive/MyDrive/AI Basics with Tensorflow/CATS-v-DOGS/cats-v-dogs/testing/dogs/"

split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

# Expected output
# 666.jpg is zero length, so ignoring
# 11702.jpg is zero length, so ignoring

666.jpg is zero length, so ignoring
11702.jpg is zero length, so ignoring


In [41]:
print(len(os.listdir('/content/drive/MyDrive/AI Basics with Tensorflow/CATS-v-DOGS/cats-v-dogs/training/cats/')))
print(len(os.listdir('/content/drive/MyDrive/AI Basics with Tensorflow/CATS-v-DOGS/cats-v-dogs/training/dogs/')))
print(len(os.listdir('/content/drive/MyDrive/AI Basics with Tensorflow/CATS-v-DOGS/cats-v-dogs/testing/cats/')))
print(len(os.listdir('/content/drive/MyDrive/AI Basics with Tensorflow/CATS-v-DOGS/cats-v-dogs/testing/dogs/')))

# Expected output:
# 11250
# 11250
# 1250
# 1250

11245
11231
1255
1269


In [3]:
# DEFINE A KERAS MODEL TO CLASSIFY CATS V DOGS
# USE AT LEAST 3 CONVOLUTION LAYERS
model = tf.keras.models.Sequential([
    ### START CODE HERE
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    
    ### END CODE HERE
])

model.compile(optimizer=RMSprop(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [18]:
TRAINING_DIR = '/content/drive/MyDrive/AI Basics with Tensorflow/CATS-v-DOGS/cats-v-dogs/training' #YOUR CODE HERE

train_datagen = ImageDataGenerator(
      rescale = 1./255,
	    rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')#YOUR CODE HERE

train_generator = train_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(150,150),
	class_mode='binary',
  batch_size=126
)#YOUR CODE HERE

VALIDATION_DIR = '/content/drive/MyDrive/AI Basics with Tensorflow/CATS-v-DOGS/cats-v-dogs/testing' #YOUR CODE HERE

validation_datagen = ImageDataGenerator(rescale = 1./255)#YOUR CODE HERE
validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=(150,150),
	class_mode='binary',
  batch_size=126
)#YOUR CODE HERE



# Expected Output:
# Found 22498 images belonging to 2 classes.
# Found 2500 images belonging to 2 classes.

Found 22475 images belonging to 2 classes.
Found 2523 images belonging to 2 classes.


Note: You can ignore the `UserWarning: Possibly corrupt EXIF data.` warnings.

In [27]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 74, 74, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 72, 72, 64)        36928     
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 36, 36, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_18 (Conv2D)          (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 17, 17, 128)     

In [2]:
#Save Model Checkpoint to Google Drive During Training
checkpoint_path = "/content/drive/My Drive/AI Basics with Tensorflow/CATS-v-DOGS/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
history = model.fit(train_generator,
                              epochs=15,
                              verbose=1,
                              validation_data=validation_generator,
                              callbacks = [cp_callback])

# The expectation here is that the model will train, and that accuracy will be > 95% on both training and validation
# i.e. acc:A1 and val_acc:A2 will be visible, and both A1 and A2 will be > .9

Epoch 1/15
 96/179 [===============>..............] - ETA: 1:54 - loss: 0.7577 - accuracy: 0.5320

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only

179/179 [==============================] - ETA: 0s - loss: 0.7256 - accuracy: 0.5467
Epoch 00001: saving model to /content/drive/My Drive/AI Basics with Tensorflow/CATS-V-DOGS/cp.ckpt
179/179 [==============================] - 273s 1s/step - loss: 0.7256 - accuracy: 0.5467 - val_loss: 0.6520 - val_accuracy: 0.6056
Epoch 2/15
179/179 [==============================] - ETA: 0s - loss: 0.6524 - accuracy: 0.6238
Epoch 00002: saving model to /content/drive/My Drive/AI Basics with Tensorflow/CATS-V-DOGS/cp.ckpt
179/179 [==============================] - 260s 1s/step - loss: 0.6524 - accuracy: 0.6238 - val_loss: 0.5608 - val_accuracy: 0.7170
Epoch 3/15
179/179 [==============================] - ETA: 0s - loss: 0.6068 - accuracy: 0.6720
Epoch 00003: saving model to /content/drive/My Drive/AI Basics with Tensorflow/CATS-V-DOGS/cp.ckpt
179/179 [==============================] - 263s 1s/step - loss: 0.6068 - accuracy: 0.6720 - val_loss: 0.5387 - val_accuracy: 0.7376
Epoch 4/15
179/179 [==========

In [ ]:
# PLOT LOSS AND ACCURACY
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")


plt.title('Training and validation loss')

# Desired output. Charts with training and validation metrics. No crash :)

In [21]:
checkpoint_path = "/content/drive/My Drive/AI Basics with Tensorflow/CATS-v-DOGS/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

model.load_weights(checkpoint_path)

eval_hist = model.evaluate(validation_generator)

21/21 [==============================] - 13s 597ms/step - loss: 0.4194 - accuracy: 0.7994


**Important Note:** Due to some compatibility issues, the following code block will result in an error after you select the images(s) to upload if you are running this notebook as a `Colab` on the `Safari` browser. For `all other broswers`, continue with the next code block and ignore the next one after it.

The ones running the `Colab` on `Safari`, comment out the code block below, uncomment the next code block and run it.

In [16]:
# Here's a codeblock just for fun. You should be able to upload an image here 
# and have it classified without crashing

import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150,150,3))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a dog")
  else:
    print(fn + " is a cat")

Saving cat.100.jpg to cat.100.jpg
Saving cat.105.jpg to cat.105.jpg
Saving cat-g7ca36d26b_1920.jpg to cat-g7ca36d26b_1920.jpg
Saving cat-g9da85a0e4_1920.jpg to cat-g9da85a0e4_1920.jpg
Saving dog.110.jpg to dog.110.jpg
Saving dog.2005.jpg to dog.2005.jpg
Saving dog-g9bd984686_1920.jpg to dog-g9bd984686_1920.jpg
[0.]
cat.100.jpg is a cat
[0.]
cat.105.jpg is a cat
[0.]
cat-g7ca36d26b_1920.jpg is a cat
[0.]
cat-g9da85a0e4_1920.jpg is a cat
[6.0907774e-07]
dog.110.jpg is a cat
[1.]
dog.2005.jpg is a dog
[1.]
dog-g9bd984686_1920.jpg is a dog


For those running this `Colab` on `Safari` broswer can upload the images(s) manually. Follow the instructions, uncomment the code block below and run it.

Instructions on how to upload image(s) manually in a Colab:

1. Select the `folder` icon on the left `menu bar`.
2. Click on the `folder with an arrow pointing upwards` named `..`
3. Click on the `folder` named `tmp`.
4. Inside of the `tmp` folder, `create a new folder` called `images`. You'll see the `New folder` option by clicking the `3 vertical dots` menu button next to the `tmp` folder.
5. Inside of the new `images` folder, upload an image(s) of your choice, preferably of either a horse or a human. Drag and drop the images(s) on top of the `images` folder.
6. Uncomment and run the code block below. 

In [ ]:
# import numpy as np
# from keras.preprocessing import image
# import os

# images = os.listdir("/tmp/images")

# print(images)

# for i in images:
#  print()
#  # predicting images
#  path = '/tmp/images/' + i
#  img = image.load_img(path, target_size=(150, 150))
#  x = image.img_to_array(img)
#  x = np.expand_dims(x, axis=0)

#  images = np.vstack([x])
#  classes = model.predict(images, batch_size=10)
#  print(classes[0])
#  if classes[0]>0.5:
#    print(i + " is a dog")
#  else:
#    print(i + " is a cat")